# Exporting METS and PAGE XML files from a Transkribus collection

In [ ]:
import json
import re

with open('./issue-parser-result.json') as issue_json:
  issue_desc = json.load(issue_json)

to_export_transkribus = issue_desc['document-id']
#to_export_transkribus = 3289764 #4580229

collectionId = re.search(r"\((\w+)\)", issue_desc['source-collection']).group(0) [1:-1]

#collectionId = 795736

print(to_export_transkribus)
print(collectionId)

## Setup

In [ ]:
collId = collectionId
docId = to_export_transkribus

In [ ]:
!pip install lxml_html_clean
!pip install lxml[html_clean]
!pip install requests-toolbelt

import requests
from requests_toolbelt.multipart.encoder import MultipartEncoder
import os
from lxml import etree
import json

In [ ]:
if secretsPath:
    with open(secretsPath, 'r') as secretsFile:
        secrets = json.loads(secretsFile.read())
        for (k, v) in secrets.items():
            os.environ[k] = v

creds = json.loads(os.environ["TRANSKRIBUS_CREDENTIALS_SIMLER"])

s = requests.Session()
s.post('https://transkribus.eu/TrpServer/rest/auth/login', data=creds)

## Get and transform METS, get PAGE XML

<!-- does it make sense to get mets files, too? 

http://transkribus.eu/TrpServer/rest/collections/{collId}/{id}/mets

-->

In [ ]:
docs = s.get('https://transkribus.eu/TrpServer/rest/collections/'+ str(collId) +'/'+ str(docId) +'/mets')

# Parse the METS XML content
try:
  tree = etree.fromstring(docs.content)
  tree_unaltered = etree.fromstring(docs.content)

  mets_xml = etree.tostring(tree, pretty_print=True).decode('utf-8')

  # Find all xlink:href attributes in the namespace
  for element in tree.xpath("//*[@*[local-name()='href' and namespace-uri()='http://www.w3.org/1999/xlink']]"):
      href_attribute = element.get('{http://www.w3.org/1999/xlink}href')
      
      # Find the parent ns3:file element
      parent_file = element.getparent()
      while parent_file is not None and parent_file.tag != '{http://www.loc.gov/METS/}file':
          parent_file = parent_file.getparent()

      if parent_file is not None:
          file_id = parent_file.get('ID')
          if file_id:
            match = re.search(r'(\d+)', file_id)
            if match:
              numeric_id = match.group(1)
              new_href = "page/" + f"{numeric_id.zfill(4)}_p" + f"{numeric_id.zfill(3)}.xml" # Replace with your desired pattern using file_id if needed
              element.set('{http://www.w3.org/1999/xlink}href', new_href)
          else:
              print(f"Warning: Parent ns3:file element has no ID attribute for element: {etree.tostring(element)}")
      else:
          print(f"Warning: No parent ns3:file element found for element: {etree.tostring(element)}")

  print(etree.tostring(tree, pretty_print=True).decode())

  mets_xml = etree.tostring(tree, pretty_print=True).decode('utf-8')

  with open("./scratch/mets.xml", "w", encoding="utf-8") as f:
    f.write(mets_xml)

  # Use XPath to find all ns3:file elements with mime type application/xml
  namespace = {'ns2': 'http://www.w3.org/1999/xlink', 'ns3': 'http://www.loc.gov/METS/'}
  for file_element in tree_unaltered.xpath("//ns3:file[@MIMETYPE='application/xml']", namespaces=namespace):
      # Find the ns2:href attribute within the ns3:FLocat child element
      href = file_element.xpath("./ns3:FLocat/@ns2:href", namespaces=namespace)

      id_attribute = file_element.xpath("./@ID")
      if id_attribute:
          # Extract the numeric part of the ID
          match = re.search(r'(\d+)', id_attribute[0])
          if match:
              numeric_id = match.group(1)
              # Format the file name
              formatted_filename = "scratch/page/" + f"{numeric_id.zfill(4)}_p" + f"{numeric_id.zfill(3)}.xml"
              print(f"ID: {id_attribute[0]}, Formatted filename: {formatted_filename}")
          else:
              print(f"Warning: Could not extract numeric ID from {id_attribute[0]}")
      else:
          print("Warning: ID attribute not found for an element.")

      if href:
          xml_url = href[0]
          try:
              xml_response = s.get(xml_url)
              xml_response.raise_for_status() # Raise an exception for bad status codes (4xx or 5xx)
              xml_tree = etree.fromstring(xml_response.content)
              print(etree.tostring(xml_tree, pretty_print=True).decode())
              print(formatted_filename)
              with open(formatted_filename, 'w', encoding="utf-8") as xml_file:
                  xml_file.write(etree.tostring(xml_tree, pretty_print=True).decode('utf-8'))

          except requests.exceptions.RequestException as e:
              print(f"Error fetching XML from {xml_url}: {e}")
          except etree.XMLSyntaxError as e:
              print(f"Error parsing XML from {xml_url}: {e}")
      else:
          print("ns2:href attribute not found in ns3:FLocat element.")

except etree.XMLSyntaxError as e:
  print(f"Error parsing XML: {e}")
  print(f"Response content: {docs.content}")